# Version 3 for Infant 1 processing
1. Read the data
1. Perform interquartile range smoothing for outlier
1. Normalize the data to (-1, 1)
1. Calculate the Heart rate (BPM) and respiration rate (BPM)
    - save the data into .npy files
1. Resample the data to same cardinality
1. Correlation test
1. Linear Regression
    - export model
1. Polynomial Regression
    - export model
1. Support Vector Regression (SVR)
    - export model

In [ ]:
# Import of libraries/modules
### -------------------------------------------
import wfdb
from wfdb import processing
### ------------------------------------------- 
import scipy
from scipy.signal import butter, lfilter, filtfilt
### -------------------------------------------
import matplotlib.pyplot as plt
### -------------------------------------------
import numpy as np
### -------------------------------------------
import pandas as pd
### -------------------------------------------
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib
### -------------------------------------------
import functools
### -------------------------------------------
import seaborn as sns
### -------------------------------------------
scaler = MinMaxScaler(feature_range=(-1,1))
data_dir = "../data"

In [5]:
# Some Custom Function
def startTime_num(time, fs):
    n = time * fs
    return n


def iqr_remove_outlier(x, lower, upper):
    if (x < lower):
        return lower
    elif (x > upper): 
        return upper
    else:
        return x


def correlationTest(signal_1, signal_2, plot=True):
    # Inspect by scatter plot
    if plot: 
        plt.scatter(signal_1, signal_2)
    # Covariance
    covariance = np.cov(signal_1, signal_2)
    print(covariance)
    # calculate Pearson's correlation - 0 is no correlation -1 or 1 is highly correlated
    corr, _ = scipy.stats.pearsonr(signal_1, signal_2)
    print('Pearsons correlation: %.3f' % corr)
    # calculate spearman's correlation - 0 is no correlation -1 or 1 is highly correlated
    corr, _ = scipy.stats.spearmanr(signal_1, signal_2)
    print('Spearmans correlation: %.3f' % corr)


def peaks_hr(sig, peak_inds, fs, title, figsize=(20, 10), saveto=None):
    "Plot a signal with its peaks and heart rate"
    # Calculate heart rate
    hrs = processing.hr.compute_hr(sig_len=sig.shape[0], qrs_inds=peak_inds, fs=fs)
    
    N = sig.shape[0]
    
    fig, ax_left = plt.subplots(figsize=figsize)
    ax_right = ax_left.twinx()
    
    ax_left.plot(sig, color='#3979f0', label='Signal')
    ax_left.plot(peak_inds, sig[peak_inds], 'rx', marker='x', 
                 color='#8b0000', label='Peak', markersize=12)
    ax_right.plot(np.arange(N), hrs, label='Heart rate', color='m', linewidth=2)

    ax_left.set_title(title)

    ax_left.set_xlabel('Time (ms)')
    ax_left.set_ylabel('ECG (mV)', color='#3979f0')
    ax_right.set_ylabel('Heart rate (bpm)', color='m')
    # Make the y-axis label, ticks and tick labels match the line color.
    ax_left.tick_params('y', colors='#3979f0')
    ax_right.tick_params('y', colors='m')
    if saveto is not None:
        plt.savefig(saveto, dpi=600)
    plt.show()


def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band', analog=False)
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a


def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y


def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a


def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_highpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y


def peaks_rr(sig, peak_inds, fs, title, figsize=(20, 10), saveto=None):
    "Plot a signal with its peaks and heart rate"
    # Calculate heart rate
    hrs = processing.hr.compute_hr(sig_len=sig.shape[0], qrs_inds=peak_inds, fs=fs)
    
    N = sig.shape[0]
    
    fig, ax_left = plt.subplots(figsize=figsize)
    ax_right = ax_left.twinx()
    
    ax_left.plot(sig, color='#3979f0', label='Signal')
    ax_left.plot(peak_inds, sig[peak_inds], 'rx', marker='x', 
                 color='#8b0000', label='Peak', markersize=12)
    ax_right.plot(np.arange(N), hrs, label='Repiration rate', color='m', linewidth=2)

    ax_left.set_title(title)

    ax_left.set_xlabel('Time (ms)')
    ax_left.set_ylabel('RESP (NU)', color='#3979f0')
    ax_right.set_ylabel('Repiration rate (bpm)', color='m')
    # Make the y-axis label, ticks and tick labels match the line color.
    ax_left.tick_params('y', colors='#3979f0')
    ax_right.tick_params('y', colors='m')
    if saveto is not None:
        plt.savefig(saveto, dpi=600)
    plt.show()

### Read the data

In [7]:
# Read all the data
signal_ecg_0 = wfdb.rdsamp(f"{data_dir}/infant1_ecg")
record_ecg_0 = wfdb.rdrecord(f"{data_dir}/infant1_ecg")
dataframe_ecg_0 = record_ecg_0.to_dataframe()
annotation_ecg_0 = wfdb.rdann(f"{data_dir}/infant1_ecg", 'qrsc',shift_samps=True)
startNum_ECG = 0
endNum_ECG = signal_ecg_0[0].shape[0]

signal_resp_0 = wfdb.rdsamp(f"{data_dir}/infant1_resp")
record_resp_0 = wfdb.rdrecord(f"{data_dir}/infant1_resp")
annotation_resp_0 = wfdb.rdann(f"{data_dir}/infant1_resp", 'resp',shift_samps=True)
dataframe_resp_0 = record_resp_0.to_dataframe()
startNum_RESP = 0
endNum_RESP = signal_resp_0[0].shape[0]

In [9]:
startTime_seconds_ECG = startNum_ECG/record_ecg_0.fs
endTime_seconds_ECG = endNum_ECG/record_ecg_0.fs
startTime_minutes_ECG = (startTime_seconds_ECG)/60
endTime_minutes_ECG = (endTime_seconds_ECG)/60
startTime_hours_ECG = startTime_minutes_ECG/60
endTime_hours_ECG = endTime_minutes_ECG/60

In [10]:
startTime_seconds_RESP = startNum_RESP/record_resp_0.fs
endTime_seconds_RESP = endNum_RESP/record_resp_0.fs
startTime_minutes_RESP = (startTime_seconds_RESP)/60
endTime_minutes_RESP = (endTime_seconds_RESP)/60
startTime_hours_RESP = startTime_minutes_RESP/60
endTime_hours_RESP = endTime_minutes_RESP/60

In [14]:
# Generate Timestamp array - ECG
start = startTime_seconds_ECG
stop = endTime_seconds_ECG
step = 1/record_ecg_0.fs
time_array_seconds_ECG = np.arange(start=start, stop=stop, step=step)
time_array_minutes_ECG = time_array_seconds_ECG/60

In [15]:
# Generate Timestamp array - RESP
start = startTime_seconds_RESP
stop = endTime_seconds_RESP
step = 1/record_resp_0.fs
time_array_seconds_RESP = np.arange(start=start, stop=stop, step=step)
time_array_minutes_RESP = time_array_seconds_RESP/60

### Interquartile (IQR) Smoothing

In [11]:
# ECG DATA - Description with IQR
print("ECG DATA")
print("-----------------------------------------------------------------------------------")
print(signal_ecg_0[1])
print(signal_ecg_0[0].shape)
print(f"Start time: {startTime_seconds_ECG} seconds, End time: {endTime_seconds_ECG} seconds")
print(f"Start time: {startTime_minutes_ECG} minutes, End time: {endTime_minutes_ECG} minutes")
print(f"Start time: {startTime_hours_ECG} hours, End time: {endTime_hours_ECG} hours")
print(dataframe_ecg_0.describe())
q75_ECG, q25_ECG = np.percentile(signal_ecg_0[0], [75, 27])
q90_ECG, q10_ECG = np.percentile(signal_ecg_0[0], [90, 10])
q95_ECG, q5_ECG = np.percentile(signal_ecg_0[0], [95, 5])
print(f"25th percentile: {q25_ECG}, 75th percentile: {q75_ECG}")
print(f"10th percentile: {q10_ECG}, 90th percentile: {q90_ECG}")
print(f"5th percentile: {q5_ECG}, 95th percentile: {q95_ECG}")

ECG DATA
-----------------------------------------------------------------------------------
{'fs': 250, 'sig_len': 41052191, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['mV'], 'sig_name': ['ECG'], 'comments': []}
(41052191, 1)
Start time: 0.0 seconds, End time: 164208.764 seconds
Start time: 0.0 minutes, End time: 2736.812733333333 minutes
Start time: 0.0 hours, End time: 45.613545555555554 hours
                ECG
count  4.105219e+07
mean  -1.884517e-01
std    2.565324e+00
min   -4.095997e+01
25%   -1.273949e-01
50%   -5.245674e-02
75%    8.742790e-03
max    4.089003e+01
25th percentile: -0.11740318639741702, 75th percentile: 0.008742790476403396
10th percentile: -0.20857800136562385, 90th percentile: 0.05870159319870851
5th percentile: -0.2497940136115256, 95th percentile: 0.3172383972866375


In [16]:
# RESP DATA - Description with IQR 
print("RESP DATA")
print("-----------------------------------------------------------------------------------")
print(signal_resp_0[1])
print(signal_resp_0[0].shape)
print(f"Start time: {startTime_seconds_RESP} seconds, End time: {endTime_seconds_RESP} seconds")
print(f"Start time: {startTime_minutes_RESP} minutes, End time: {endTime_minutes_RESP} minutes")
print(f"Start time: {startTime_hours_RESP} hours, End time: {endTime_hours_RESP} hours")
print(dataframe_resp_0.describe())
q75_RESP, q25_RESP = np.percentile(signal_resp_0[0], [75, 27])
q90_RESP, q10_RESP = np.percentile(signal_resp_0[0], [90, 10])
q95_RESP, q5_RESP = np.percentile(signal_resp_0[0], [95, 5])
print(f"25th percentile: {q25_RESP}, 75th percentile: {q75_RESP}")
print(f"10th percentile: {q10_RESP}, 90th percentile: {q90_RESP}")
print(f"5th percentile: {q5_RESP}, 95th percentile: {q95_RESP}")

RESP DATA
-----------------------------------------------------------------------------------
{'fs': 500, 'sig_len': 82122000, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['NU'], 'sig_name': ['RESP'], 'comments': []}
(82122000, 1)
Start time: 0.0 seconds, End time: 164244.0 seconds
Start time: 0.0 minutes, End time: 2737.4 minutes
Start time: 0.0 hours, End time: 45.623333333333335 hours
               RESP
count  8.212200e+07
mean   2.429525e+01
std    1.876535e+00
min   -5.579915e-01
25%    2.388994e+01
50%    2.417714e+01
75%    2.460235e+01
max    4.832892e+01
25th percentile: 23.933954698105357, 75th percentile: 24.602350889649504
10th percentile: 23.373725300951726, 90th percentile: 25.36175638406015
5th percentile: 22.89779140563346, 95th percentile: 26.255437899093508


In [17]:
# Interquartile 
iqr1_ECG = q75_ECG-q25_ECG
iqr2_ECG = q90_ECG-q10_ECG
iqr3_ECG = q95_ECG-q5_ECG
decimal_ECG = 3
print(f"ECG")
print(f"Percentiles: 25th={q25_ECG:.{decimal_ECG}f}, 75th={q75_ECG:.{decimal_ECG}f}, IQR={iqr1_ECG:.{decimal_ECG}f}")
print(f"Percentiles: 10th={q10_ECG:.{decimal_ECG}f}, 90th={q90_ECG:.{decimal_ECG}f}, IQR={iqr2_ECG:.{decimal_ECG}f}")
print(f"Percentiles: 5th={q5_ECG:.{decimal_ECG}f}, 95th={q95_ECG:.{decimal_ECG}f}, IQR={iqr3_ECG:.{decimal_ECG}f}")
print("--------------------------------")
iqr1_RESP = q75_RESP-q25_RESP
iqr2_RESP = q90_RESP-q10_RESP
iqr3_RESP = q95_RESP-q5_RESP
decimal_RESP = 3
print(f"RESP")
print(f"Percentiles: 25th={q25_RESP:.{decimal_RESP}f}, 75th={q75_RESP:.{decimal_RESP}f}, IQR={iqr1_RESP:.{decimal_RESP}f}")
print(f"Percentiles: 10th={q10_RESP:.{decimal_RESP}f}, 90th={q90_RESP:.{decimal_RESP}f}, IQR={iqr2_RESP:.{decimal_RESP}f}")
print(f"Percentiles: 5th={q5_RESP:.{decimal_RESP}f}, 95th={q95_RESP:.{decimal_RESP}f}, IQR={iqr3_RESP:.{decimal_RESP}f}")

ECG
Percentiles: 25th=-0.117, 75th=0.009, IQR=0.126
Percentiles: 10th=-0.209, 90th=0.059, IQR=0.267
Percentiles: 5th=-0.250, 95th=0.317, IQR=0.567
--------------------------------
RESP
Percentiles: 25th=23.934, 75th=24.602, IQR=0.668
Percentiles: 10th=23.374, 90th=25.362, IQR=1.988
Percentiles: 5th=22.898, 95th=26.255, IQR=3.358


In [18]:
# ECG DATA - Remove Outlier
# calculate the outlier cutoff
print("ECG DATA")
cut_off_ECG = iqr1_ECG * 1.5
lower_ECG, upper_ECG = q25_ECG - cut_off_ECG, q75_ECG + cut_off_ECG

# identify outliers
signals_ECG_outliers = [x for x in signal_ecg_0[0] if x < lower_ECG or x > upper_ECG]
print('Identified outliers: %d' % len(signals_ECG_outliers))

signal_ecg_1 = []
signal_ecg_1 = map(functools.partial(iqr_remove_outlier, lower=lower_ECG, upper=upper_ECG), signal_ecg_0[0])
print(signal_ecg_1)
signal_ecg_1 = np.fromiter(signal_ecg_1, dtype=np.float64)
print(f"ECG Data Shape: {signal_ecg_1.shape}")
print(signal_ecg_1)

ECG DATA
Identified outliers: 3490340
ECG Data Shape: (41052191,)
[-0.01998352 -0.01998352 -0.01498764 ... -0.30662215 -0.07868511
  0.19796176]


In [21]:
# RESP DATA - Remove Outlier
# calculate the outlier cutoff
print("RESP DATA")
cut_off_RESP = iqr1_RESP * 1
lower_RESP, upper_RESP = q25_RESP - cut_off_RESP, q75_RESP + cut_off_RESP
# identify outliers
signals_RESP_outliers = [x for x in signal_resp_0[0] if x < lower_RESP or x > upper_RESP]
print('Identified outliers: %d' % len(signals_ECG_outliers))
signal_resp_1 =[]
signal_resp_1 = map(functools.partial(iqr_remove_outlier, lower=lower_RESP, upper=upper_RESP), signal_resp_0[0])
print(signal_resp_1)
signal_resp_1 = np.fromiter(signal_resp_1, dtype=np.float64)
print(f"RESP Data Shape: {signal_resp_1.shape}")
print(signal_resp_1)

RESP DATA
Identified outliers: 3490340
RESP Data Shape: (82122000,)
[23.26555851 23.26555851 23.26555851 ... 24.17565154 24.17714349
 24.17266762]


### Data Normalization (-1,1)

In [22]:
# Reshape the data
print(signal_ecg_1.shape)
signal_ecg_2 = signal_ecg_1.reshape((-1,1))
print(signal_ecg_2.shape)
print(signal_resp_1.shape)
signal_resp_2 = signal_resp_1.reshape((-1,1))
print(signal_resp_2.shape)

(41052191,)
(41052191, 1)
(82122000,)
(82122000, 1)


In [23]:
# Normalization
signal_ecg_3 = scaler.fit_transform(signal_ecg_2)
signal_resp_3 = scaler.fit_transform(signal_resp_2)

### Heartrate and Respiration Rate Calculation

In [24]:
ECG_qrs_inds_1 = processing.qrs.gqrs_detect(sig=signal_ecg_3, fs=record_ecg_0.fs)
ECG_hrs_1 = processing.hr.compute_hr(sig_len=signal_ecg_3.shape[0], qrs_inds=ECG_qrs_inds_1, fs=record_ecg_0.fs)
RESP_peaks_inds_1 = processing.peaks.find_local_peaks(sig=signal_resp_3, radius=record_resp_0.fs)
RESP_rrs_1 = processing.hr.compute_hr(sig_len=signal_resp_3.shape[0], qrs_inds=RESP_peaks_inds_1, fs=record_resp_0.fs)

KeyboardInterrupt: 

In [ ]:
np.save(file='../data/infant_1_hrs_1.npy', arr=ECG_hrs_1)
np.save(file='../data/infant_1_rrs_1.npy', arr=RESP_rrs_1)

### Data Resampling

In [ ]:
RESP_rrs_2 = pd.DataFrame(RESP_rrs_1).fillna(0).to_numpy().reshape(RESP_rrs_1.shape[0])
ECG_hrs_2 = pd.DataFrame(ECG_hrs_1).fillna(0).to_numpy().reshape(ECG_hrs_1.shape[0])
print(RESP_rrs_2.shape)
print(ECG_hrs_2.shape)

In [ ]:
# Resample the data to same cardinality - downsampling
if (RESP_rrs_2.shape[0] < ECG_hrs_2.shape[0]):
    ECG_hrs_3 = scipy.signal.resample(ECG_hrs_2, RESP_rrs_2.shape[0])
    RESP_rrs_3 = RESP_rrs_2
else:
    RESP_rrs_3 = scipy.signal.resample(RESP_rrs_2, ECG_hrs_2.shape[0])
    ECG_hrs_3 = ECG_hrs_2

### Correlation Test

In [ ]:
correlationTest(ECG_hrs_3, RESP_rrs_3, plot=False)

### Linear Regression

### Polynomial Regression

### Support Vector Regression (SVR)